### Dict example using trait
* Weakness of ***OOP***
  * same data, but requires new data type for each different functionality

In [19]:
import scala.annotation.tailrec
trait Iter[A] {
    def getValue: Option[A]
    def getNext: Iter[A]
}
trait Dict[K, V] {
    def add(k: K, v: V): Dict[K ,V]
    def find(k: K): Option[V]
}
// java style interface
class ListIterDict[K, V](eq: (K, K)=>Boolean, list: List[(K, V)])
    extends Iter[(K, V)] with Dict[K, V] {
    def getValue = list.headOption
    def getNext = new ListIterDict(eq, list.tail)
    
    def add(k: K, v: V) = new ListIterDict(eq, (k, v) :: list)
    def find(k: K) = {
        @tailrec
        def findIter(l: List[(K, V)]): Option[V] ={
            l match {
                case Nil => None
                case (key, v) :: tl if eq(key, k) => Some(v)
                case _ :: tl => findIter(tl)
            }
        }
        findIter(list)
    }
    def printDict = {
        @tailrec
        def printDictIter(l: List[(K, V)]): Option[V] ={
            l match {
                case Nil => None
                case kv :: tl => {
                    print(kv)
                    printDictIter(tl)
                }
            }
        }
        printDictIter(list)
    }
}

def sumElements[A](f: A=>Int)(xs: Iter[A]): Int = {
    xs.getValue match {
        case None => 0
        case Some(n) => f(n) + sumElements(f)(xs.getNext)
    }
}

import scala.annotation.tailrec

defined trait Iter
defined trait Dict
defined class ListIterDict
defined function sumElements

In [22]:
val myDict = new ListIterDict[Int, String]((x,y)=>x==y, Nil)
val newDict = myDict.add(4, "four").add(3, "tree").add(2, "two").add(1, "one").add(3, "three")
newDict.printDict
newDict.find(3)
sumElements[(Int, String)](x=>x._1)(newDict)

(3,three)(1,one)(2,two)(3,tree)(4,four)

myDict: ListIterDict[Int, String] = ammonite.$sess.cmd18$Helper$ListIterDict@67cf8484
newDict: ListIterDict[Int, String] = ammonite.$sess.cmd18$Helper$ListIterDict@4b5fdff5
res21_2: Option[String] = None
res21_3: Option[String] = Some("three")
res21_4: Int = 13

### Mixin with Traits

In [33]:
abstract class Iter[A] {
    def getValue: Option[A]
    def getNext: Iter[A]
}
class ListIter[A](list: List[A]) extends Iter[A] {
    def getValue = list.headOption
    def getNext: Iter[A] = new ListIter(list.tail)
}

defined class Iter
defined class ListIter

In [36]:
trait MRIter[A] extends Iter[A] {
    // if not? error
    override def getNext: MRIter[A]
    def mapReduce[B,C](combine: (B,C)=>C, ival: C, f: A=>B): C = {
        getValue match {
            case None => ival
            case Some(v) => combine(f(v), getNext.mapReduce(combine, ival, f))
        }
    }
}

defined trait MRIter

In [34]:
class MRListIter[A](list: List[A]) extends ListIter[A](list) with MRIter[A]

cmd34.sc:1: incompatible type in overriding
override def getNext: cmd34.this.cmd33.MRIter[A] (defined in trait MRIter)
  with def getNext: cmd34.this.cmd32.Iter[A] (defined in class ListIter);
 found   : cmd34.this.cmd32.Iter[A]
 required: cmd34.this.cmd33.MRIter[A]
class MRListIter[A](list: List[A]) extends ListIter[A](list) with MRIter[A]
      ^Compilation Failed

: 

#### Solution 1 (worst)

In [39]:
class MRListIter[A](list: List[A]) extends ListIter[A](list) with MRIter[A] {
    override def getNext = new MRListIter(list.tail)
}

defined class MRListIter

#### Solution 2

In [44]:
class ListIter[A](val list: List[A]) extends Iter[A] {
    def getValue = list.headOption
    def getNext: ListIter[A] = new ListIter(list.tail)
}
class MRListIter[A](list: List[A]) extends ListIter[A](list) with MRIter[A] {
    override def getNext = new MRListIter(super.getNext.list)
}

defined class ListIter
defined class MRListIter

#### Solution 3

In [47]:
trait MRIter[A] extends Iter[A] {
    def mapReduce[B,C](combine: (B,C)=>C, ival: C, f: A=>B): C = {
        def mapReduceIter(xs: Iter[A]): C = {
            xs.getValue match {
                case None => ival
                case Some(v) => combine(f(v), mapReduceIter(xs.getNext))
            }
        }
        mapReduceIter(this)
    }
}
class MRListIter[A](list: List[A]) extends ListIter[A](list) with MRIter[A]

defined trait MRIter
defined class MRListIter

In [57]:
val mr1 = new MRListIter[Int](List(3,4,5))
mr1.mapReduce[Int,Int]((b,c)=>b+c,0,(a)=>a*a)

// syntactic sugar
// what is type of mr2? intersection type
val mr2 = new ListIter[Int](List(3,4,5)) with MRIter[Int]
mr2.mapReduce[Int,Int]((b,c)=>b+c,0,(a)=>a*a)

mr1: MRListIter[Int] = ammonite.$sess.cmd46$Helper$MRListIter@508a303c
res56_1: Int = 50
mr2: ListIter[Int] with MRIter[Int] = ammonite.$sess.cmd56$Helper$$anon$1@63096b5b
res56_3: Int = 50